## Лабораторна робота №2: "Імплементація алгоритмів стиснення"

Склад команди та розподіл виконаних завдань:

-
-

Для кожного з алгоритмів поданих нижче
- опишіть як працює алгорит
- напишіть класи з методами encode та decode
- перевірте правильність кодування та декодування
- дослідіть час виконання коду в залежності від розмірів вхідних даних
- оцініть ступінь стиснення(у відсотка) в залежності від розмірів
- напишіть висновок про ефективність різних алгоритмів та умови за яких той чи інший алгоритм дають кращий результат

In [28]:
class Node:
    def __init__(self, left, right=None, freq=0) -> None:
        self.left = left
        self.right = right
        self.freq = freq
    def __repr__(self) -> str:
        return f"Node(left:{self.left}, right:{self.right}, freq:{self.freq})"

def tree_proc(tree, code_dct, code=''):
    left_branch = (tree.left, code + '0')
    right_branch = (tree.right, code + '1')
    if isinstance(left_branch[0], str) or isinstance(left_branch[0], tuple):
        code_dct[left_branch[0]] = left_branch[1][:-1]
    elif isinstance(left_branch[0], Node):
        tree_proc(left_branch[0], code_dct, code+'0')
    if isinstance(right_branch[0], str) or isinstance(right_branch[0], tuple):
        code_dct[left_branch] = right_branch[1]
    elif isinstance(right_branch[0], Node):
        tree_proc(right_branch[0], code_dct, code+'1')
    return code_dct

In [32]:
class Huffman:
    def encode(self, text: str) -> tuple[str, dict[str, str]]:
        freq_lst = []
        code_dict = {}
        encoded_msg = ''
        for char in set(text):
            freq_lst.append((char, text.count(char)))
        freq_lst = sorted(freq_lst, key=lambda x: x[-1])
        freq_lst = [Node(x[0], freq=x[1]) for x in freq_lst]
        while len(freq_lst) > 2:
            freq_lst = sorted(freq_lst, key=lambda x:x.freq)
            freq_lst.append(Node(freq_lst[0], freq_lst[1], freq_lst[0].freq + freq_lst[1].freq))
            freq_lst.pop(0)
            freq_lst.pop(0)
        tree = Node(freq_lst[0], freq_lst[1], None)
        code_dict = tree_proc(tree, code_dict)
        for ch in text:
            encoded_msg += f'{code_dict[ch]}'
        return (encoded_msg, code_dict)

    def decode(self, code: str, coding_dict: dict[str, str], for_lz:bool=False):
        temp_code = ''
        decoded_msg = ''
        decoding_dct = {x:y for y, x in coding_dict.items()}
        for el in code:
            temp_code += el
            if temp_code in decoding_dct.keys():
                if not for_lz:
                    decoded_msg += decoding_dct[temp_code]
                else:
                    decoded_msg += f"{decoding_dct[temp_code]};"
                temp_code = ''
            else:
                continue
        return decoded_msg

if __name__ == '__main__':
    coder = Huffman()
    msg = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Fusce a quam laoreet, eleifend magna scelerisque, dignissim elit. Praesent sodales nisi vehicula, finibus tortor sit amet, auctor ex. Vivamus congue dapibus ipsum eget varius. Mauris tempor erat et nisi dictum laoreet. In imperdiet porta diam sed fringilla. Donec hendrerit sollicitudin nunc ac fermentum. Donec nibh urna, congue eu facilisis quis, dignissim a lectus. Praesent id dolor vitae massa egestas efficitur non at orci. Praesent dapibus dui quis turpis luctus vehicula. Ut placerat purus eget auctor tempor.'
    encoded_msg = coder.encode(msg)
    print(encoded_msg)
    print(coder.decode(encoded_msg[0], encoded_msg[1]) == msg)
    print(coder.decode(encoded_msg[0], encoded_msg[1]))


('01010100111000100001011111100001111001010011001111110010111110010110111000100110101000010011101000011110011001010100110111011110011111101000111101100100110010110010011010000101100011110000010101110100011111101110110001101100001001101111110011101011011010101110100111010001100101011011010000111111010110100011100010000100110010101001100011011000100011110110011111101011110011111000101110111111000110101011101001101100010100000101001010110110001010100110010110001011101111100010101010000011111100011011000010011011111101111010101001000001101000111111100111010101110001011100010110001101011011111000101000011001110110010111001000111010110101101000010100110111101100011111000011100001101010110100111100010010011110001001101010000100111010000111100110010101001101000011011101100111100010011000111110100010111111001110100000001110111000011110110101011011101111001111110111001100011100101110001111000000111000011010101100001111001010011001111110001101110001100111001110111000010000001101010101111110011101

# Алгоритм LZW

In [19]:
class LZW:
    def encode(self, text: str) -> tuple[str, list]:
        coding_dict = {}
        encoded_msg = []
        last_ind = 0
        ind = 0
        while ind != len(text):
            ch = text[ind]
            if ch in coding_dict and ind != len(text)-1:
                enc = f'{ch}'
                cur_ind = ind
                for ad in text[ind+1:]:
                    cur_ind += 1
                    old_enc = enc
                    enc += ad
                    if enc not in coding_dict:
                        encoded_msg.append(coding_dict[old_enc])
                        coding_dict[enc] = str(last_ind + 1)
                        last_ind += 1
                        ind = cur_ind
                        break
            elif ch in coding_dict and ind == len(text)-1:
                encoded_msg.append(coding_dict[ch])
                ind += 1
            else:
                coding_dict[ch] = str(last_ind + 1)
                last_ind += 1
                ind += 1
                encoded_msg.append(coding_dict[ch])
        return (','.join(encoded_msg), coding_dict.items())

    def decode(self, code: str, coding_dict: list) -> str:
        res = ''
        dict_of_codes = {key:value for (value, key) in coding_dict}
        for num in code.split(','):
            res += dict_of_codes[num]
        return res

lzw_coder = LZW()
original_msg = 'CRAZY? I WAS CRAZY ONCE... THEY LOCKED ME IN A ROOM, A RUBBER ROOM... A RUBBER ROOM WITH RATS AND RATS MADE ME CRAZY... '
encoded = lzw_coder.encode(original_msg)
decoded = lzw_coder.decode(encoded[0], encoded[1])
res = (encoded, decoded)
print(original_msg == decoded)
print(res)

True
(('1,2,3,4,5,6,7,8,7,10,3,12,7,1,2,3,4,5,7,20,21,1,23,24,24,24,7,28,29,23,18,32,20,1,35,23,37,7,39,23,7,8,21,7,3,7,2,20,20,39,51,44,46,54,55,55,23,2,46,48,39,25,26,45,2,54,55,56,58,47,49,9,8,28,29,46,3,28,12,44,21,37,76,78,38,3,37,40,39,40,14,16,5,62,7', dict_items([('C', '1'), ('R', '2'), ('A', '3'), ('Z', '4'), ('Y', '5'), ('?', '6'), (' ', '7'), ('I', '8'), (' W', '9'), ('W', '10'), ('AS', '11'), ('S', '12'), (' C', '13'), ('CR', '14'), ('RA', '15'), ('AZ', '16'), ('ZY', '17'), ('Y ', '18'), (' O', '19'), ('O', '20'), ('N', '21'), ('CE', '22'), ('E', '23'), ('.', '24'), ('..', '25'), ('. ', '26'), (' T', '27'), ('T', '28'), ('H', '29'), ('EY', '30'), ('Y L', '31'), ('L', '32'), ('OC', '33'), ('CK', '34'), ('K', '35'), ('ED', '36'), ('D', '37'), (' M', '38'), ('M', '39'), ('E ', '40'), (' I', '41'), ('IN', '42'), ('N ', '43'), (' A', '44'), ('A ', '45'), (' R', '46'), ('RO', '47'), ('OO', '48'), ('OM', '49'), ('M,', '50'), (',', '51'), (' A ', '52'), (' RU', '53'), ('U', '54'), 

# Алгоритм LZ77

Потрібно заміряти розміри саме тексту, проте для роботи доцільно використовувати список тюплів, тому для зручності варто імплементувати додаткові алгоритми _text2list та _list2text

In [20]:
class LZ77:
    def __init__(self, buffer_size: int):
        self.buffer_size = buffer_size

    def encode(self, text: str) -> str:
        matched=""
        encoded=[]
        ind=0
        while ind<len(text):
            element=text[ind]
            if element not in matched:
                matched+=element
                encoded.append((0,0,element))
                ind+=1
            else:
                while element in matched[-(self.buffer_size//2):]:
                    ind+=1
                    element+=text[ind]
                if len(element)>2:
                    for i,el in enumerate(matched[::-1]):
                        if el==element[-2]:
                            ofset=i+1+len(element[:-2])
                            matchi=len(element[:-1])
                            left=text[ind]
                            encoded.append((ofset,matchi,left))
                            matched+=element
                            ind=len(matched)
                            break
                else:
                    for i,el in enumerate(matched[::-1]):
                        if el==element[0]:
                            ofset=i+1
                            matchi=1
                            left=text[ind]
                            encoded.append((ofset,matchi,left))
                            matched+=element
                            ind+=len(element)-1
                            break
        return encoded
    
    def decode(self, code: str) -> str:
        decoded=''
        proses=''
        for cod in code:
            matches=cod[1]
            ofset=cod[0]
            i=0
            while i<matches:
                proses+=decoded[-ofset+i]
                i+=1
            proses+=cod[2]
            decoded+=proses
            proses=""
        return decoded

In [40]:
def test():
    lz = LZ77(buffer_size=10)
    text = "abacababacabc"
    encoded=lz.encode(text)
    assert lz.encode(text) == [(0, 0, 'a'), (0, 0, 'b'), (2, 1, 'c'), (4, 3, 'b'), (2, 1, 'c'), (4, 2, 'c')]
    assert lz.decode(encoded) == "abacababacabc"

if __name__== '__main__':
    test()
    print("All assertions passed successfully!")

All assertions passed successfully!


# Алгоритм Deflate

In [22]:
def from_huffman_to_str(huffman_encoded):
    res = ''
    res += f'{huffman_encoded[0]};'
    res += '|'.join(f"{x}:{y}"for x, y in huffman_encoded[1].items())
    return res

def from_str_to_huffman(msg):
    text = msg.split(';')[0]
    str_lst = [t.split(":") for t in msg.split(';')[1].split('|')]
    dct = {x:y for x, y in str_lst}
    return (text, dct)

In [27]:
class Deflate:
    def __init__(self, buffer_size: int):
        self.buffer_size = buffer_size

    def encode(self, text: str) -> str:
        lz_encoded = LZ77(self.buffer_size).encode(text)
        fully_encoded = Huffman().encode(lz_encoded)
        return from_huffman_to_str(fully_encoded)

    def decode(self, code: str) -> str:
        code = from_str_to_huffman(code)
        huffman_decoded = Huffman().decode(code[0], code[1], True)
        huffman_decoded = huffman_decoded.split(';')[:-1]
        ready_to_be_fully_dec = []
        for tup in huffman_decoded:
            off = int(tup.replace('(', '').replace(')', '').split(',')[0])
            ind = int(tup.replace('(', '').replace(')', '').split(',')[1])
            ch = tup.replace('(', '').replace(')', '').split(',')[2].replace("'", '').strip()
            ready_to_be_fully_dec.append((off, ind, ch))
        fully_decoded = LZ77(self.buffer_size).decode(ready_to_be_fully_dec)
        return fully_decoded

In [39]:
if __name__ == '__main__':
    deflator = Deflate(10)
    msg = 'abacababacabcabacababacabcabacababacabcaba cababacabcabacababacabcabacababacabcabacababacabcabacababacabcabacababacabc'
    deflated_msg = deflator.encode(msg)
    flated_msg = deflator.decode(deflated_msg)
    print(deflated_msg)
    print(msg == flated_msg)
    print(flated_msg)

110011101010110111100100111100100111100100110110110001001001111001001111001001111001001111001001111001;(3, 2, 'a'):00|(4, 2, 'c'):01|(2, 1, 'c'):10|(5, 4, 'b'):11000|(0, 0, 'a'):11001|(0, 0, 'b'):11010|(0, 0, ' '):110110|(4, 3, 'b'):110111|(4, 4, 'b'):111
False
abacababacabcabacababacabcabacababacabcababcabbbcbbcbbacbbabacabcabacababacabcabacababacabcabacababacabcabacababacabc
